In [1]:
import cv2
from cv2 import dnn
import tensorflow as tf
from tensorflow.python.framework.convert_to_constants import convert_variables_to_constants_v2
from tensorflow import keras
import matplotlib.pyplot as plt
%matplotlib inline
import numpy as np
import glob


In [2]:
def convert_h5to_pb():
    path=r'C:/Users/Administrator/Desktop/my_cnn/断点/6月25日.h5'
    model = tf.keras.models.load_model(path,compile=False)
    model.summary()
    full_model = tf.function(lambda Input: model(Input))
    full_model = full_model.get_concrete_function(tf.TensorSpec(model.inputs[0].shape, model.inputs[0].dtype))
    # Get frozen ConcreteFunction
    frozen_func = convert_variables_to_constants_v2(full_model)
    frozen_func.graph.as_graph_def()
    layers = [op.name for op in frozen_func.graph.get_operations()]
    print("-" * 50)
    print("Frozen model layers: ")
    for layer in layers:
        print(layer)
    print("-" * 50)
    print("Frozen model inputs: ")
    print(frozen_func.inputs)
    print("Frozen model outputs: ")
    print(frozen_func.outputs)
    pbpath=r'C:\Users\Administrator\Desktop\my_cnn\断点'
    # Save frozen graph from frozen ConcreteFunction to hard drive
    tf.io.write_graph(graph_or_graph_def=frozen_func.graph,
                      logdir=pbpath,
                      name="全6月24号90精度.pb",
                      as_text=False)
convert_h5to_pb()
print('finish')

Model: "sequential_7"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
my_iuput (Conv2D)            (None, 78, 78, 64)        640       
_________________________________________________________________
batch_normalization_59 (Batc (None, 78, 78, 64)        256       
_________________________________________________________________
conv2d_57 (Conv2D)           (None, 76, 76, 64)        36928     
_________________________________________________________________
batch_normalization_60 (Batc (None, 76, 76, 64)        256       
_________________________________________________________________
max_pooling2d_17 (MaxPooling (None, 38, 38, 64)        0         
_________________________________________________________________
conv2d_58 (Conv2D)           (None, 36, 36, 128)       73856     
_________________________________________________________________
batch_normalization_61 (Batc (None, 36, 36, 128)      

finish


In [180]:
path=r'C:\Users\Administrator\Desktop\my_cnn\pbmodel\all88.6.pb'
net = dnn.readNetFromTensorflow(path)
help(dnn.blobFromImage)


Help on built-in function blobFromImage:

blobFromImage(...)
    blobFromImage(image[, scalefactor[, size[, mean[, swapRB[, crop[, ddepth]]]]]]) -> retval
    .   @brief Creates 4-dimensional blob from image. Optionally resizes and crops @p image from center,
    .        *  subtract @p mean values, scales values by @p scalefactor, swap Blue and Red channels.
    .        *  @param image input image (with 1-, 3- or 4-channels).
    .        *  @param size spatial size for output image
    .        *  @param mean scalar with mean values which are subtracted from channels. Values are intended
    .        *  to be in (mean-R, mean-G, mean-B) order if @p image has BGR ordering and @p swapRB is true.
    .        *  @param scalefactor multiplier for @p image values.
    .        *  @param swapRB flag which indicates that swap first and last channels
    .        *  in 3-channel image is necessary.
    .        *  @param crop flag which indicates whether image will be cropped after resize o

In [67]:
help(net.forward)

Help on built-in function forward:

forward(...) method of cv2.dnn_Net instance
    forward([, outputName]) -> retval
    .   @brief Runs forward pass to compute output of layer with name @p outputName.
    .            *  @param outputName name for layer which output is needed to get
    .            *  @return blob for first output of specified layer.
    .            *  @details By default runs forward pass for the whole network.
    
    
    
    forward([, outputBlobs[, outputName]]) -> outputBlobs
    .   @brief Runs forward pass to compute output of layer with name @p outputName.
    .            *  @param outputBlobs contains all output blobs for specified layer.
    .            *  @param outputName name for layer which output is needed to get
    .            *  @details If @p outputName is empty, runs forward pass for the whole network.
    
    
    
    forward(outBlobNames[, outputBlobs]) -> outputBlobs
    .   @brief Runs forward pass to compute outputs of layers listed

In [ ]:
imgpath=r'C:\Users\Administrator\Desktop\叫.png'
img = cv2.imdecode(np.fromfile(imgpath, dtype=np.uint8),cv2.IMREAD_UNCHANGED)#读取
#img = cv2.imdecode(np.fromfile(imgpath, dtype=np.uint8),cv2.COLOR_BGR2GRAY)
img = cv2.cvtColor(img,cv2.COLOR_BGR2GRAY)#灰度化
plt.imshow(img)
#img = np.array(img)   
#img=np.expand_dims(img, axis=2)
inputBlob=dnn.blobFromImage(img, scalefactor=0.00390625,size=(70,70), swapRB=False,crop=False,ddepth = cv2.CV_32F )#预处理
#print(img.shape)
#img=cv2.convertScaleAbs(img,cv2.CV_32F,1)
#img=img/255


#plt.imshow(inputBlob)
#dnn.blobFromImage(img, scalefactor=1, swapRB=True,crop=False,ddepth = cv2.CV_32F )

#img = np.array(img)   
#img=np.expand_dims(img, axis=0)
#img=np.expand_dims(img, axis=3)
#print(img.shape)
#inputBlob = inputBlob.reshape(0,70,70,1)
#inputBlob=np.transpose(inputBlob,[1,2,3,0])#改变维度
print(inputBlob.shape)

#net.setInput(inputBlob,name='Input')#set the network input, "data" is the name of the input layer
#net.setInput(img)
net.setInput(inputBlob)
#net.getMemoryShapes
#pred=np.zeros(shape=(3745,1))

pred=net.forward()
print(pred)
np.argmax(pred)